# Week 09

## Setup

Run the following 2 cells to import all necessary libraries and helpers for this week's exercises

In [ ]:
!wget -q https://github.com/DM-GY-9103-2024S-R/9103-utils/raw/main/src/io_utils.py

In [ ]:
import numpy as np
import pandas as pd

## Audio

- load
- play
- get samples

## Images

- load
- show
- get samples
- manipulate